# Introduction

1. Load in the collected data and camera parameters
2. Reformat the collected data for easier viewing and analysis
3. Calculate the projection matrices




## Load in the data

Begin by building a dictionary that consolidates all of the calibration data collected.

In [1]:

import numpy as np
import json
from pathlib import Path

cam0_cal = "calibration_params\cam_0.json"
cam1_cal = "calibration_params\cam_1.json"
stereocam_0_1_cal = "calibration_params\stereo_cam_0_cam_1.json"

calibration_data = {
                    "cam0": cam0_cal, 
                    "cam1": cam1_cal, 
                    "stereocam": stereocam_0_1_cal
                    }


for key, json_path in calibration_data.items():
    with open(Path(Path.cwd(), json_path)) as f:
        calibration_data[key] = json.load(f)


## Reconfigure the data

For ease of processing and plotting, convert the observed 2D image data into a consolidated dataframe.

In [2]:
# trying a different way to get a list of lists in the condition that I want

import pandas as pd

imgpointsA = calibration_data["stereocam"]["image_points_A"]
imgpointsB = calibration_data["stereocam"]["image_points_B"]
imgpointsID = calibration_data["stereocam"]["image_points_ID"]

frame = 0
summarized_points = []

for ID, pointA, pointB in zip(imgpointsID, imgpointsA, imgpointsB):
    for i, uv_A, uv_B in zip(ID, pointA, pointB):
        summarized_points.append([frame, int(i), uv_A[0], uv_A[1], uv_B[0], uv_B[1]])

    # print(frame)
    frame = frame + 1

df_pnts = pd.DataFrame(summarized_points,
                        columns = ["frame", "ID", "u_A", "v_A", "u_B", "v_B"])



## Define the Direct Linear Transformation to 3D Space
Isolate the parameters needed to create the projection matrices. Use them to define the Direct Linear Transformation that will be used to map the pair of 2D images to a 3D estimate. 

In [3]:

# cam0 orientation will define the global frame of reference, so
# RT1 is this...not fully sure what this stuff means
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
mtx1 = calibration_data["cam0"]["camera_matrix"]
P1 = mtx1 @ RT1 #projection matrix for C1

R2 = calibration_data["stereocam"]["rotation_vector"]
T2 = calibration_data["stereocam"]["translation_vector"]
RT2 = np.concatenate([R2, T2], axis = -1)
mtx2 = calibration_data["cam1"]["camera_matrix"]
P2 = mtx2 @ RT2 #projection matrix for C2


# here u and v are used by convention to indicate the x,y position of the detected point in a 2D plane.
def triangulate(P1, P2, u_A, v_A, u_B, v_B):
 
    point1 = np.array([u_A, v_A])
    point2 = np.array([u_B, v_B])
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    return Vh[3,0:3]/Vh[3,3]


# Estimate 3D Position

Traverse the dataframe and apply the DLT to each pair of points using the projection matrices calculated above

In [4]:

# see: https://stackoverflow.com/a/63832539/13563258
# this is a madlad answer right here

df_pnts[["X", "Y", "Z"]] = [triangulate(P1, P2, *a) for a in tuple(zip(df_pnts["u_A"], df_pnts["v_A"],df_pnts["u_B"],df_pnts["v_B"]))]
df_pnts.to_csv("tracked_points.csv")


# Inspect results for reasonability



In [35]:
import plotly.express as px


fig = px.scatter_3d(df_pnts, x='X', y='Y', z='Z',
              color='frame')
fig.show()

Things appear to be sitting in a plane nicely, even when viewed more closely for just a single frame.

In [36]:
import plotly.express as px

single_board = df_pnts[df_pnts["frame"]==3]

fig = px.scatter_3d(single_board, x='X', y='Y', z='Z', color='ID')
fig.show()

Previously I was having some issues with image distortion, but this does not appear to be the case after recalibrating. Now I will compare the distances between points in order to assess the 3d accuracy of the point estimate. 

See tutorial [here](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html) for details regarding how to undistort an image. I suspect that this may be something where it is preferable to go back to the source (i.e. perform a new calibration).

# Calculate Distance Between Calibration Corners



In [39]:
from charuco import Charuco

charuco = Charuco(4,5,11,8.5, aruco_scale=.75, square_size_overide=.0525)
connected_corners = charuco.get_connected_corners()

connected_corners

adjacent_corners = [(x,y) for x,y in connected_corners if abs(x-y)==1]

def distance(ID_A, ID_B, frame):

    ptA_3d = (
    df_pnts
    .query('frame == ' + str(frame))
    .query('ID == ' + str(ID_A))
    .filter(["X", "Y", "Z"])
    ).to_numpy()

    ptB_3d = (
    df_pnts
    .query('frame == ' + str(frame))
    .query('ID == ' + str(ID_B))
    .filter(["X", "Y", "Z"])
    ).to_numpy()

    dist = np.linalg.norm(ptA_3d- ptB_3d)
    return dist

distances = []

for frame in range(0,df_pnts["frame"].max()):
    for pair in adjacent_corners:
        distances.append(distance(pair[0], pair[1], 8))

distances = pd.DataFrame(distances, columns = ["Distances"])
print("Mean distance between corners is ", distances.mean())
print("Standard deviation between corners is", distances.std())

Mean distance between corners is  Distances    0.053826
dtype: float64
Standard deviation between corners is Distances    0.00013
dtype: float64


OK. I'm feeling pretty good about where these calibration numbers are landing. It appears to be within 2 mm of the gold standard realitY measuring stick. An important part of any of this is going to involve some kind of visualization of markers in space, so I might as well figure out how to do that now. Also, I could fairly rapidly create an animation of the board moving through space to post to the Discord, and I feel like I might need that now.



# 3D Plotting


I originally tried working with plotly and then looked at matplotlib for 3D plotting options. Both left me with a sense that they were not the right tool for the job. I got excited by ipyvolume, but then began to have some trouble with just the basic installation of it. I will try again...


The above is not working. Go back to the tutorial example to get clarity on the format that the inputs must take.

https://ipyvolume.readthedocs.io/en/latest/examples/animation.html

It appears that it requires X,Y,Z coordinates to be converted into arrays. This will certainly be a happy thing for quickly integrating the raw output of the triangulate function, though currently presents a challenge for working with the numpy data. 

Ok. I am seeing something that looks like it might present a solution to the problem I'm dealing with:

*Can ipyvolume do a plot that update with real time data?*

https://github.com/widgetti/ipyvolume/issues/235

Below is a method that will work for displaying sequential point data *as it comes in*, which is quite interesting. There are more native ways within ipyvolume for playing back data, but it will require plugging of missing data.

In [33]:
# df_ipv = df_pnts.query("frame == 4")
import ipywidgets as widgets
import ipyvolume as ipv
from time import sleep
import keyboard

view_not_shown = True
# set range of bounding cube

max_x = df_pnts["X"].max()
max_y = df_pnts["Y"].max()
max_z = df_pnts["Z"].max()

min_x = df_pnts["X"].min()
min_y = df_pnts["Y"].min()
min_z = df_pnts["Z"].min()

range_x = max_x - min_x
range_y = max_y - min_y
range_z = max_z - min_z

max_range = max(range_x, range_y, range_z)

for f in range(0, df_pnts["frame"].max()):

    if view_not_shown:
        
        df_ipv = df_pnts.query(f"frame == 1")
        x = df_ipv["X"].to_numpy()
        y = df_ipv["Y"].to_numpy()
        z = df_ipv["Z"].to_numpy()

        fig = ipv.figure()

        ipv.pylab.xlim(min_x, min_x + max_range)
        ipv.pylab.ylim(min_y, min_y + max_range)
        ipv.pylab.zlim(min_z, min_z + max_range)
        
        fig.animation = 0
        scatter = ipv.scatter(x, y, z)
        ipv.show()
        view_not_shown = False

    df_ipv = df_pnts.query(f"frame == {f}")

    x = df_ipv["X"].to_numpy()
    y = df_ipv["Y"].to_numpy()
    z = df_ipv["Z"].to_numpy()

    scatter.x = x
    scatter.y = y
    scatter.z = z

    sleep(.5)

    if keyboard.is_pressed("q"):
        print("q pressed, ending loop")
        break




Attempt to do the above, but within the actual constraints of `ipyvolume` to use the native animate functionality. This will require some larger reformatting of the dataframe. And if I accomplish this, I think it could be a useful thing.

In [32]:
# Get all the IDs observed in the capture session

IDs = df_pnts["ID"].unique()
frames = df_pnts["frame"].unique()

# print(IDs)
# checks out

x_points = []
y_points = []
z_points = []

for frame in frames:
    x_frame_points  = []
    y_frame_points  = []
    z_frame_points  = []
    subset_frame = df_pnts.query(f"frame == {frame}")

    for id in ID:
        # build a list of all the ID values:
        subset_ID = subset_frame.query(f"ID == {id}")
        # print(subset_ID["ID"].unique())
        if subset_ID.empty:
            x_frame_points.append(0)
            # x_points.append
            print("empty")
    x_points.append(x_frame_points)

print(x_points)
    



TypeError: 'bool' object is not callable

# Now What?

I think I may have hit the moment where I really need to just look at capturing real data and displaying it. Then I could post it and call it a day. Kinda sad that DO is going to the night birds thing, but then I know that this is something she needs to do and we will have our evening time tonight. Plus, I may be getting to a good place right now with the process. 